# Sklearnモデルのデプロイ
推論Dockerイメージを作成し、Azure Container Instancesにデプロイします。

## Python SDK バージョン確認

In [1]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.10


## Workspace情報の取得

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

exp = Experiment(workspace = ws, name = "devops-notebook")

Falling back to use azure cli credentials. This fall back to use azure cli credentials will be removed in the next release. 
Make sure your code doesn't require 'az login' to have happened before using azureml-sdk, except the case when you are specifying AzureCliAuthentication in azureml-sdk.


Found the config file in: /Users/konabuta/Project/DevOpsLab/code/aml_config/config.json
Workspace name: azureml
Azure region: southeastasia
Subscription id: 9c0f91b8-eb2f-484c-979c-15848c098a6b
Resource group: azureml_sea


## モデル取得

In [3]:
from azureml.core.model import Model
model=Model(ws, 'diabetes-notebook')

## 推論コードの準備

In [14]:
!pwd

/Users/konabuta/Project/DevOpsLab/code/notebook


In [7]:
import os 
script_folder = "scoring"
os.makedirs(script_folder, exist_ok=True)

In [8]:
%%writefile {script_folder}/score_diabetes.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('diabetes-notebook')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Overwriting scoring/score_diabetes.py


## condaパッケージの指定

In [9]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open(script_folder+"/myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [10]:
with open(script_folder+"/myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- scikit-learn



## 推論Dockerイメージ構築

### Dockerイメージ設定

In [15]:
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

os.chdir("scoring")

# 推論Dockerイメージの設定
image_config = ContainerImage.image_configuration(execution_script="score_diabetes.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

### Dockerイメージ作成

In [16]:
%%time
from azureml.core.image import Image,ContainerImage

image = Image.create(name = "sklearn-diabete-image",
                     models = [model],
                     image_config = image_config,
                     workspace = ws)

image.wait_for_creation(show_output = True)


Creating image
Running............................
SucceededImage creation operation finished for image sklearn-diabete-image:2, operation "Succeeded"
CPU times: user 1.33 s, sys: 97.7 ms, total: 1.42 s
Wall time: 3min 10s


In [18]:
os.chdir("..")

In [19]:
#print('{}(v.{} [{}]) stored at {} with build log {}'.format(image.name, image.version, image.creation_state, image.image_location, image.image_build_log_uri))

## 推論Dockerイメージのデプロイ

In [23]:
%%time
from azureml.core.webservice import AciWebservice
import datetime

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=1, 
                                            tags={'area': "diabetes", 'type': "regression"},
                                            description='A diabetes model from jupyter')

service = Webservice.deploy_from_image(deployment_config=aciconfig,
                                        image=image,
                                        name="diabetes"+ datetime.datetime.now().strftime('%m%d%H%M'),
                                        workspace=ws)

service.wait_for_deployment(show_output=True)

Creating service
Running..................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 633 ms, sys: 47.9 ms, total: 681 ms
Wall time: 1min 53s


## Webサービスのテスト

In [24]:
input_j = [[1,2,3,4,5,6,7,8,9,10],[10,9,8,7,6,5,4,3,2,1]]
print(input_j)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]]


In [25]:
import json
test_sample = json.dumps({'data': input_j})
test_sample = bytes(test_sample,encoding = 'utf8')

In [26]:
try:   
    prediction = service.run(input_data = test_sample)
    print(prediction)
except Exception as e:
    result = str(e)
    print(result)
    raise Exception('ACI service is not working as expected')

[5063.066696016179, 3703.565405644549]
